# INITIAL TABLE ASSUMPTIONS

```yaml
TABLE:
    - NAME: ZETA
    - COLS:
        - UID:
            TYPE: PRIMARY KEY
            DESC: UNIQUE ID
        - ZID:
            TYPE: INTEGER
            DESC: ZETA USER ID
        - ZDATE:
            TYPE: DATE
            DESC: ACT DATE
            NOTE: ASCENDING
```

## SET VARIABLES
*NOTE*: `DECLARE` statement is not available in `SQLite`; thus we "pseudo-assign" vars with a `TEMP TABLE`

```sqlite
CREATE TEMP TABLE IF NOT EXISTS VARS (NAME TEXT PRIMARY KEY, DVAL DATE, SVAL TEXT, IVAL INTEGER);
INSERT OR REPLACE INTO VARS (NAME, DVAL) VALUES ('FROM', '2008-01-01');
INSERT OR REPLACE INTO VARS (NAME, DVAL) VALUES ('UNTIL', '2008-05-01');
INSERT OR REPLACE INTO VARS (NAME, SVAL, IVAL) VALUES ('CHURN', '+28 days', 28);
-- SELECT * FROM VARS
```

## TABLE WITH THE DATES OF INTEREST

```sqlite
CREATE TEMP TABLE IF NOT EXISTS ZETA2 AS
	SELECT ZT.*
	FROM ZETA AS ZT
	WHERE ZT.ZDATE
		BETWEEN
			(SELECT DVAL FROM VARS WHERE NAME = 'FROM')
		AND
			(SELECT DVAL FROM VARS WHERE NAME = 'UNTIL');
-- SELECT * FROM ZETA2
```

# QUERIES

### 1. NEW USERS PER DAY

```sqlite
WITH NEW AS (
--	DATE (ASC): GROUP BY GIVES 1ST DATE PER USER
	SELECT UID
	FROM ZETA2
	GROUP BY ZID
)
-- SUM NEW USERS PER DAY
SELECT	ZT.ZDATE AS "DATE",
		SUM(CASE
			WHEN ZT.UID IN (SELECT * FROM NEW) THEN 1
			ELSE 0
			END) AS "NEW USERS"
FROM ZETA2 AS ZT
GROUP BY ZT.ZDATE;
```

### 2. CHURN ACCOUNTS PER DAY

```sqlite
WITH
UNTIL AS (
--	TO IMPROVE READABILITY
	SELECT DATE(DVAL, '-1 days') AS DVAL
	FROM VARS
	WHERE NAME = 'UNTIL'
	LIMIT 1
),
CHURN AS (
--	TO IMPROVE READABILITY
	SELECT IVAL, SVAL FROM VARS
	WHERE NAME = 'CHURN'
	LIMIT 1
),
DATES AS (
--	GET CHURN DATES
	SELECT	ZT.ZDATE,
			CASE
			WHEN
				--	IF ZID IS THE SAME AND DATE DIFFERENCE IS OVER CHURN
					(ZT.ZID = LEAD(ZT.ZID) OVER (ORDER BY ZT.ZID)
					AND
					JULIANDAY(LEAD(ZT.ZDATE) OVER (ORDER BY ZT.ZID)) - JULIANDAY(ZT.ZDATE) >= (SELECT IVAL FROM CHURN))
				OR
				--	IF LAST ZID AND DATE DIFFERENCE UNTIL END IS OVER CHURN
					(ZT.ZID <> LEAD(ZT.ZID) OVER (ORDER BY ZT.ZID)
					AND
					JULIANDAY((SELECT * FROM UNTIL)) - JULIANDAY(ZT.ZDATE) >= (SELECT IVAL FROM CHURN))
			THEN DATE(ZT.ZDATE, (SELECT SVAL FROM CHURN))
			ELSE NULL
			END AS CDATE
	FROM ZETA2 AS ZT
)
--	COUNT CHURN USERS PER DAY
SELECT	DD.ZDATE AS "DATE",
		CASE
		--	IF DATE IS IN CHURN DATES
		WHEN DD.ZDATE IN (SELECT K.CDATE FROM DATES K)
		THEN (SELECT COUNT(CDATE) FROM DATES K WHERE CDATE = DD.ZDATE)
		ELSE 0
		END AS "USERS CHURN"
FROM DATES AS DD
GROUP BY DD.ZDATE;
```

### 3. REACTIVATED ACCOUNTS PER DAY

```sqlite
WITH
_CHURN AS (
--	TO IMPROVE READABILITY
	SELECT IVAL FROM VARS
	WHERE NAME = 'CHURN'
),
_REACT AS (
--	DAYS WITH REACTIVATIONS
	SELECT	ZDATE,
			CASE
			WHEN
			--	IF ZIDS ARE EQUAL AND THE DAY DIFF IS OVER CHURN
				(ZID = LAG(ZID) OVER (ORDER BY ZID))
				AND
				(JULIANDAY(ZDATE) - JULIANDAY(LAG(ZDATE) OVER (ORDER BY ZID)) >= (SELECT * FROM _CHURN))
			THEN 1
			ELSE 0
			END AS DIFF
	FROM ZETA2
)
--	SUM REACTIVATIONS PER DAY
SELECT	ZDATE AS "DATE",
		SUM(DIFF) AS "USERS REACTIVATED"
FROM _REACT
GROUP BY ZDATE;
```